<a href="https://colab.research.google.com/github/fatimasadiq/NLP_Module_Project_hatespeechdetection/blob/main/NLP_Project_FromSpring_dataset_task2_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing Libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
import string
string.punctuation
import re
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Data Loading 
df=pd.read_csv('/content/FromSpring_datasets.csv' , error_bad_lines=False, delimiter=",") #error_bad_lines=False usecols=["label","q_a"]
#df['label']=df['label'].astype('object')
df.head(5)

,posts,label
0,I like too many songs to have a favorite,0
1,</3 ? haha jk! <33,0
2,Really?!?! Thanks?! haha,0
3,;(,0
4,*RAWR*?,0


In [ ]:
#text lower
df["posts"] = df["posts"].str.lower()

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
df['posts']= df['posts'].apply(lambda x:remove_punctuation(x))

#removing numbers 
def remove_numbers(text):
  output=''.join(c for c in text if not c.isdigit())
  return output
#applying the function
#text=this person 100 fake beware isabella
df['posts']= df['posts'].apply(lambda x:remove_numbers(x))
df.head(5)

,posts,label
0,i like too many songs to have a favorite,0
1,haha jk,0
2,really thanks haha,0
3,,0
4,rawr,0


In [ ]:
# LR and TFIDF after removing punctuations and numbers

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score ,classification_report ,accuracy_score

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['posts'], df['label'] , test_size=0.2, random_state=42)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
print('train posts length: ',len(train_x))
print('test posts length: ',len(valid_x))

train posts length:  10180
test posts length:  2546


In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(df['posts'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['posts'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['posts'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['posts'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    #print(metrics.classification_report(valid_y, predictions, target_names=['q_a', 'label']))
    f1score=metrics.f1_score(predictions,valid_y , average='weighted')
    accuracy=metrics.accuracy_score(predictions, valid_y)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return accuracy, f1score

In [ ]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR Accuracy, WordLevel TF-IDF: ", accuracy)
print("LR F1score, WordLevel TF-IDF: ", f1score)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR Accuracy, N-Gram Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

# Linear Classifier on Character Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR Accuracy, CharLevel Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

LR Accuracy, WordLevel TF-IDF:  0.9544383346425765
LR F1score, WordLevel TF-IDF:  0.9631488109815419
LR Accuracy, N-Gram Vectors:  0.9375490966221524
LR F1score, N-Gram Vectors:  0.9638774351120077
LR Accuracy, CharLevel Vectors:  0.9560094265514533
LR F1score, N-Gram Vectors:  0.9639191483409262


In [ ]:
#Data Loading 
df=pd.read_csv('/content/FromSpring_datasets.csv' , error_bad_lines=False, delimiter=",") #error_bad_lines=False usecols=["label","q_a"]
#df['label']=df['label'].astype('object')
df.head(5)

,posts,label
0,I like too many songs to have a favorite,0
1,</3 ? haha jk! <33,0
2,Really?!?! Thanks?! haha,0
3,;(,0
4,*RAWR*?,0


In [ ]:
print(len(df))

12726


In [ ]:
#removing stop words
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in stopwords])

df["posts"] = df["posts"].apply(lambda text: remove_stopwords(text))
df.head()

,q_a,label
0,tell guy gay seem super straight,0
1,Idk I'm gay,0
2,This person 100% fake beware Isabella!,1
3,Oof lmao fake? Who's lmao shit fuck speak like...,1
4,If loved one left n cut ties w u wait bc I sti...,0


In [ ]:
#Accuracy and F1score after stopwords removal

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score ,classification_report ,accuracy_score

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['posts'], df['label'] , test_size=0.2, random_state=42)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
print('train posts length: ',len(train_x))
print('test posts length: ',len(valid_x))

train posts length:  10180
test posts length:  2546


In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(df['posts'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['posts'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['posts'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['posts'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    #print(metrics.classification_report(valid_y, predictions, target_names=['q_a', 'label']))
    f1score=metrics.f1_score(predictions,valid_y , average='weighted')
    accuracy=metrics.accuracy_score(predictions, valid_y)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return accuracy, f1score

In [ ]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR Accuracy, WordLevel TF-IDF: ", accuracy)
print("LR F1score, WordLevel TF-IDF: ", f1score)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR Accuracy, N-Gram Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

# Linear Classifier on Character Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR Accuracy, CharLevel Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

LR Accuracy, WordLevel TF-IDF:  0.9528672427336999
LR F1score, WordLevel TF-IDF:  0.963009402781837
LR Accuracy, N-Gram Vectors:  0.9375490966221524
LR F1score, N-Gram Vectors:  0.9645417287374761
LR Accuracy, CharLevel Vectors:  0.9575805184603299
LR F1score, N-Gram Vectors:  0.9654468904977279


In [ ]:
#Accuracy and F1score after Stemming

In [ ]:
#Data Loading 
df=pd.read_csv('/content/FromSpring_datasets.csv' , error_bad_lines=False, delimiter=",") #error_bad_lines=False usecols=["label","q_a"]
#df['label']=df['label'].astype('object')
df.head(5)

,posts,label
0,I like too many songs to have a favorite,0
1,</3 ? haha jk! <33,0
2,Really?!?! Thanks?! haha,0
3,;(,0
4,*RAWR*?,0


In [ ]:
#stemming
def stem_words(text):
    return " ".join([porter_stemmer.stem(word) for word in text.split()])

df["posts"] = df["posts"].apply(lambda text: stem_words(text))
df.head()

,posts,label
0,I like too mani song to have a favorit,0
1,</3 ? haha jk! <33,0
2,really?!?! thanks?! haha,0
3,;(,0
4,*rawr*?,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score ,classification_report ,accuracy_score

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['posts'], df['label'] , test_size=0.2, random_state=42)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
print('train posts length: ',len(train_x))
print('test posts length: ',len(valid_x))

train posts length:  10180
test posts length:  2546


In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(df['posts'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['posts'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['posts'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['posts'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    #print(metrics.classification_report(valid_y, predictions, target_names=['q_a', 'label']))
    f1score=metrics.f1_score(predictions,valid_y , average='weighted')
    accuracy=metrics.accuracy_score(predictions, valid_y)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return accuracy, f1score

In [ ]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR Accuracy, WordLevel TF-IDF: ", accuracy)
print("LR F1score, WordLevel TF-IDF: ", f1score)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR Accuracy, N-Gram Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

# Linear Classifier on Character Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR Accuracy, CharLevel Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

LR Accuracy, WordLevel TF-IDF:  0.9516889238020424
LR F1score, WordLevel TF-IDF:  0.9616399573092366
LR Accuracy, N-Gram Vectors:  0.9379418695993715
LR F1score, N-Gram Vectors:  0.964432745642087
LR Accuracy, CharLevel Vectors:  0.9552238805970149
LR F1score, N-Gram Vectors:  0.9643118800503233


In [ ]:
#Accuracy and F1score after applying all the preprocessing steps

In [ ]:
#Data Loading 
df=pd.read_csv('/content/FromSpring_datasets.csv' , error_bad_lines=False, delimiter=",") #error_bad_lines=False usecols=["label","q_a"]
#df['label']=df['label'].astype('object')
df.head(5)

,posts,label
0,I like too many songs to have a favorite,0
1,</3 ? haha jk! <33,0
2,Really?!?! Thanks?! haha,0
3,;(,0
4,*RAWR*?,0


In [ ]:
#by applying all the preprocessing steps at once

In [ ]:
#text lower
df["posts"] = df["posts"].str.lower()

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
df['posts']= df['posts'].apply(lambda x:remove_punctuation(x))

#removing numbers 
def remove_numbers(text):
  output=''.join(c for c in text if not c.isdigit())
  return output
#applying the function
#text=this person 100 fake beware isabella
df['posts']= df['posts'].apply(lambda x:remove_numbers(x))
df.head(5)

,posts,label
0,i like too many songs to have a favorite,0
1,haha jk,0
2,really thanks haha,0
3,,0
4,rawr,0


In [ ]:
#removing stop words
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in stopwords])

df["posts"] = df["posts"].apply(lambda text: remove_stopwords(text))
df.head()

,posts,label
0,like many songs favorite,0
1,haha jk,0
2,really thanks haha,0
3,,0
4,rawr,0


In [ ]:
dfb =df[['label' ,'posts']]
dfb.head(5)

,label,posts
0,0,like many songs favorite
1,0,haha jk
2,0,really thanks haha
3,0,
4,0,rawr


In [ ]:
print(len(dfb))

12726


In [ ]:
10180
test posts length:  2546

In [ ]:
dfb[1:2546].to_csv("springberttest.csv" , index=False)

In [ ]:
dfb[2547:10180].to_csv("sprintberttrain.csv" , index=False)

In [ ]:
#stemming
def stem_words(text):
    return " ".join([porter_stemmer.stem(word) for word in text.split()])

df["posts"] = df["posts"].apply(lambda text: stem_words(text))
df.head()

,posts,label
0,like mani song favorit,0
1,haha jk,0
2,realli thank haha,0
3,,0
4,rawr,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score ,classification_report

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['posts'], df['label'] , test_size=0.2, random_state=42)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
print('train posts length: ',len(train_x))
print('test posts length: ',len(valid_x))

train posts length:  10180
test posts length:  2546


In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(df['posts'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['posts'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['posts'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['posts'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    #print(metrics.classification_report(valid_y, predictions, target_names=['q_a', 'label']))
    f1score=metrics.f1_score(predictions,valid_y , average='weighted')
    accuracy=metrics.accuracy_score(predictions, valid_y)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return accuracy, f1score

In [ ]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR Accuracy, WordLevel TF-IDF: ", accuracy)
print("LR F1score, WordLevel TF-IDF: ", f1score)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR Accuracy, N-Gram Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

# Linear Classifier on Character Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR Accuracy, CharLevel Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

LR Accuracy, WordLevel TF-IDF:  0.9481539670070699
LR F1score, WordLevel TF-IDF:  0.9586769137424798
LR Accuracy, N-Gram Vectors:  0.9367635506677141
LR F1score, N-Gram Vectors:  0.9654896886602455
LR Accuracy, CharLevel Vectors:  0.950510604870385
LR F1score, N-Gram Vectors:  0.9608548109521645


In [ ]:
#Accuracy and F1 score after sentiment analysis

In [ ]:
#Data Loading 
df=pd.read_csv('/content/FromSpring_datasets.csv' , error_bad_lines=False, delimiter=",") #error_bad_lines=False usecols=["label","q_a"]
#df['label']=df['label'].astype('object')
df.head(5)

,posts,label
0,I like too many songs to have a favorite,0
1,</3 ? haha jk! <33,0
2,Really?!?! Thanks?! haha,0
3,;(,0
4,*RAWR*?,0


In [ ]:
#text lower
df["posts"] = df["posts"].str.lower()

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
df['posts']= df['posts'].apply(lambda x:remove_punctuation(x))

#removing numbers 
def remove_numbers(text):
  output=''.join(c for c in text if not c.isdigit())
  return output
#applying the function
#text=this person 100 fake beware isabella
df['posts']= df['posts'].apply(lambda x:remove_numbers(x))
df.head(5)

,posts,label
0,i like too many songs to have a favorite,0
1,haha jk,0
2,really thanks haha,0
3,,0
4,rawr,0


In [ ]:
#removing stop words
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in stopwords])

df["posts"] = df["posts"].apply(lambda text: remove_stopwords(text))
df.head()

,posts,label
0,like many songs favorite,0
1,haha jk,0
2,really thanks haha,0
3,,0
4,rawr,0


In [ ]:
#Sentiment Analysis
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):
# Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
# polarity_scores method of SentimentIntensityAnalyzer
# oject gives a sentiment dictionary.
# which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
    #print("Overall sentiment dictionary is : ", sentiment_dict)
    #print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
    #print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
    #print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")
    #print("Sentence Overall Rated As", end = " ")
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        return 1 #positive
    elif sentiment_dict['compound'] <= - 0.05 :
        return -1 #negative
    else :
        return 0 #neutral
    

     |████████████████████████████████| 125 kB 30.2 MB/s 


In [ ]:
# Driver code
if __name__ == "__main__" :
    print("Text Selected for VADER Sentimental Analysis :")
    sentence1 = ('''ANN is like our brain; millions and billions of cells — called neurons, which processes information in the form of electric signals. Similarly, in ANN, the network structure has an input layer, a hidden layer, and the output layer. It is also called Multi-Layer Perceptron as it has multiple layers. The hidden layer is known as a “distillation layer” that distils some critical patterns from the data/information and passes it onto the next layer. It then makes the network quicker and more productive by distinguishing the data from the data sources, leaving out the excess data.''')
    print(sentence1)
#sentiment_scores(sentence1)


Text Selected for VADER Sentimental Analysis :
ANN is like our brain; millions and billions of cells — called neurons, which processes information in the form of electric signals. Similarly, in ANN, the network structure has an input layer, a hidden layer, and the output layer. It is also called Multi-Layer Perceptron as it has multiple layers. The hidden layer is known as a “distillation layer” that distils some critical patterns from the data/information and passes it onto the next layer. It then makes the network quicker and more productive by distinguishing the data from the data sources, leaving out the excess data.


In [ ]:
df["sentiment"] = df["posts"].apply(lambda text: sentiment_scores(text))
df.head(10)

,posts,label,sentiment
0,like many songs favorite,0,1
1,haha jk,0,1
2,really thanks haha,0,1
3,,0,0
4,rawr,0,0
5,sure like tell wht u wnna know like wht use,0,1
6,emma hahahahah missssseddd yuhhhhh,0,0
7,night shuree,0,0
8,alreadi answrd,0,0
9,yes veryy trustin person may help yuh,0,1


In [ ]:
df1=df[['label' ,'sentiment']]
df1.head()

,label,sentiment
0,0,1
1,0,1
2,0,1
3,0,0
4,0,0


In [ ]:
df1['sentiment']=df1['sentiment'].astype(str)
#df1['sentiment']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df1['label']=df1['label'].astype(str)
df1['label']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0        0
1        0
2        0
3        0
4        0
        ..
12721    0
12722    1
12723    1
12724    1
12725    0
Name: label, Length: 12726, dtype: object

In [ ]:
df1[1:2000].to_csv("berttestsenti.csv" , index=False)

In [ ]:
df1[2001:10000].to_csv("berttrainsenti.csv" , index=False)

In [ ]:
df1.to_csv("askfm-sentiments.csv" , index=False)

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df1['label'], df1['sentiment'], test_size=0.2, random_state=42)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
print('train posts length: ',len(train_x))
print('test posts length: ',len(valid_x))

train posts length:  10180
test posts length:  2546


In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000, lowercase=False)
tfidf_vect.fit(df1['sentiment'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
#tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
#tfidf_vect_ngram.fit(df1['sentiment'])
#xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
#xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
#tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
#tfidf_vect_ngram_chars.fit(df1['sentiment'])
#xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
#xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    #print(metrics.classification_report(valid_y, predictions, target_names=['q_a', 'label']))
    f1score=metrics.f1_score(predictions,valid_y , average='weighted')
    accuracy=metrics.accuracy_score(predictions, valid_y)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return accuracy, f1score

In [ ]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR Accuracy, WordLevel TF-IDF: ", accuracy)
print("LR F1score, WordLevel TF-IDF: ", f1score)

# Linear Classifier on Ngram Level TF IDF Vectors
#accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
#print("LR Accuracy, N-Gram Vectors: ", accuracy)
#print("LR F1score, N-Gram Vectors: ", f1score)

# Linear Classifier on Character Level TF IDF Vectors
#accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
#print("LR Accuracy, CharLevel Vectors: ", accuracy)
#print("LR F1score, N-Gram Vectors: ", f1score)

LR Accuracy, WordLevel TF-IDF:  0.4874312647289866
LR F1score, WordLevel TF-IDF:  0.6252523601503464


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/sentiments.zip

Archive:  gdrive/My Drive/sentiments.zip
  inflating: askfm-berttrain.csv     
  inflating: __MACOSX/._askfm-berttrain.csv  
  inflating: askfm-bert-test.csv     
  inflating: __MACOSX/._askfm-bert-test.csv  


In [ ]:
!unzip gdrive/My\ Drive/wiki-news-300d-1M.vec

Archive:  gdrive/My Drive/wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [ ]:
import csv
df2=pd.read_csv('/content/wiki-news-300d-1M.vec' ,quoting=csv.QUOTE_NONE, error_bad_lines=False , delimiter=";")
#df2.head(5)

b'Skipping line 42: expected 1 fields, saw 2\n'


In [ ]:
#Data Loading 
df=pd.read_csv('/content/FromSpring_datasets.csv' , error_bad_lines=False, delimiter=",") #error_bad_lines=False usecols=["label","q_a"]
#df['label']=df['label'].astype('object')
df.head(5)

,posts,label
0,I like too many songs to have a favorite,0
1,</3 ? haha jk! <33,0
2,Really?!?! Thanks?! haha,0
3,;(,0
4,*RAWR*?,0


In [ ]:
#text lower
df["posts"] = df["posts"].str.lower()

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
df['posts']= df['posts'].apply(lambda x:remove_punctuation(x))

#removing numbers 
def remove_numbers(text):
  output=''.join(c for c in text if not c.isdigit())
  return output
#applying the function
#text=this person 100 fake beware isabella
df['posts']= df['posts'].apply(lambda x:remove_numbers(x))
df.head(5)

,posts,label
0,i like too many songs to have a favorite,0
1,haha jk,0
2,really thanks haha,0
3,,0
4,rawr,0


In [ ]:
#removing stop words
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in stopwords])

df["posts"] = df["posts"].apply(lambda text: remove_stopwords(text))
df.head()

,posts,label
0,like many songs favorite,0
1,haha jk,0
2,really thanks haha,0
3,,0
4,rawr,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score ,classification_report

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['posts'], df['label'] , test_size=0.2, random_state=42)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
print('train posts length: ',len(train_x))
print('test posts length: ',len(valid_x))

train posts length:  10180
test posts length:  2546


In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(df['posts'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['posts'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['posts'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['posts'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    #print(metrics.classification_report(valid_y, predictions, target_names=['q_a', 'label']))
    #f1score=metrics.f1_score(predictions,valid_y , average='weighted')
    #accuracy=metrics.accuracy_score(predictions, valid_y)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.f1_score(predictions,valid_y , average='weighted')

In [ ]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('/content/wiki-news-300d-1M.vec' , encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(df['posts'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
import tensorflow as tf 
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=tf.optimizers.Adam(), loss='binary_crossentropy')
    
    return model
classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("CNN, Word Embeddings",  accuracy)

319/319 [==============================] - 9s 24ms/step - loss: 0.2213
CNN, Word Embeddings 0.9667207792207791


In [ ]:
import tensorflow as tf 
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=tf.optimizers.Adam(), loss='binary_crossentropy')
    
    return model
classifier = create_cnn()
f1score = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("CNN, Word Embeddings",  f1score)

319/319 [==============================] - 8s 24ms/step - loss: 0.2299
CNN, Word Embeddings 0.9667207792207791
